# Homework \#6

## Problem 1 -- Machine Scheduling


Consider the following machine scheduling problem (MSP).  We have a set $M = \{1, 2, \ldots ,m\}$ of machines and a set $N = \{1, 2, \ldots ,n\}$ of jobs that must be performed on the machines.  Each machine $i$ has a capacity of $b_i$ units of work, and each job $j$ requires $a_{ij}$ units of work to be completed if it is scheduled on machine $i$.  All jobs must be assigned to exactly one machine.


(a) Suppose that there is a cost $c_{ij}$ for scheduling job $j \in N$ on machine $i \in M$.  There is also fixed cost of operating a machine -- that is, in order to schedule any jobs to run on machine $i \in M$, then you must pay a fixed cost of $h_i$.  Write an integer programming model to determine a least cost assignment of jobs to machines not exceeding the machine capacities in this case.  


### Answer:
$$
\begin{align*}
\text{minimize} \quad & \sum_{i \in M} \sum_{j \in N} c_{ij}x_{ij} + \sum_{i \in M} h_{i}z_{i} \\
s.t.\ & \sum_{j \in N} a_{ij}x_{ij} \leq b_{i} \quad \forall i \in M \\
& \sum_{i \in M} x_{ij} = 1 \quad \forall j \in N \\
& \sum_{j \in N} a_{ij}x_{ij} \leq b_{i}z_{i} \quad \forall i \in M \\
& x_{ij}, z_i \in \{0,1\} \quad \forall i \in M, j \in N\\
\end{align*}
$$

(b) Modify your answer to (a) to handle the following logical condition: If you operate at least 3 of the first 10 machines, and you operate at least 3 of the second 10 machines, then you must operate at most 2 of the last 10 machines.  

### Answer:

$$
\begin{align*}
\text{let} \quad &\ M_{1} = \{1,2,…,10\} \textrm{: The first 10 machines.}\\
& M_{2} = \{11,12,…,20\} \textrm{: The second 10 machines.}\\
& M_{3} = \{21,22,…,30\} \textrm{: The last 10 machines.}\\
\end{align*}
$$
$$
\begin{align*}
\text{minimize} \quad & \sum_{i \in M} \sum_{j \in N} c_{ij}x_{ij} + \sum_{i \in M} h_{i}z_{i} \\
s.t.\ & \sum_{j \in N} a_{ij}x_{ij} \leq b_{i} \quad \forall i \in M \\
& \sum_{i \in M} x_{ij} = 1 \quad \forall j \in N \\
& \sum_{j \in N} a_{ij}x_{ij} \leq b_{i}z_{i} \quad \forall i \in M \\
& \sum_{i \in M_{1}} z_i \geq 3y \\
& \sum_{i \in M_{2}} z_i \geq 3y \\
& \sum_{i \in M_{3}} z_i \leq 2 + 8(1-y) \\
& x_{ij}, z_i \in \{0,1\} \quad \forall i \in M, j \in N\\
& y \in \{0,1\}
\end{align*}
$$

(c) Now suppose that each job $j \in N$ has a duration (or length) $d_j$. The _makespan_ of a schedule is the length of time it takes to finish all the jobs.  You may remove the conditions imposed in (b). Formulate a model that will minimize the makespan of an assignment of jobs to machines not exceeding the machine capacities, and ensuring that you do not operate more than half (15) of the machines.  There is no longer a fixed or variable cost.

### Answer:
$$
\begin{align*}
\text{minimize} \quad & T \\
s.t.\ & \sum_{j \in N} d_j x_{ij} \leq T \quad \forall i \in M \\
& \sum_{j \in N} a_{ij} x_{ij} \leq b_i z_i \quad \forall i \in M \\
& \sum_{i \in M} x_{ij} = 1 \quad \forall j \in N \\
& \sum_{i \in M} z_i \leq 15 \\
& x_{ij}, z_i \in \{0, 1\} \quad \forall i \in M, j \in N \\
& T \geq 0 \\
\end{align*}
$$

(d) For each of the above problems/modifications, we will test different MIP solvers to see the effect on solution time.  For a variety of different MIP solvers (at least two; e.g., Gurobi and Cbc or HiGHS) of your choosing, solve (a), (b), and (c).

Report the CPU time and any other observations you make for at least 2 different MIP solvers.What MIP solver did you find most effective? (There isn't a "right answer" here; many factors impace MIP solvers, including how you build your model.) 

In [1]:
using Random
using JuMP, Gurobi, HiGHS, Cbc

seed = 425 # seed for data generation

N = 1:60 # jobs
M = 1:30 # machines

m = length(M)
n = length(N)

Random.seed!(seed)

# set time lengths of jobs on each machine
a = zeros(length(M),length(N))
for i in M
    for j in N
        a[i,j] = round(6+Random.rand()*Random.rand(6:25),digits=2)
    end
end

# capacity of each machine
b = Dict(zip(M,[12*sum(a[i,j] for j in N)/length(M) for i in M]))

# cost of running jobs on each machine
c = zeros(length(M),length(N))
for i in M
    for j in N
        c[i,j] = 20+round(Random.rand()*Random.rand(20:60),digits=2)
    end
end

# fixed cost of each machine
h = [30*length(M) for i in M]

# duration of each job (for question (c))
d = [Random.rand(1:10)*Random.rand()+10 for j in N];

### Answer: test (a) 

In [2]:
mod = Model(Gurobi.Optimizer)
set_optimizer_attribute(mod, "OutputFlag", 0)

@variable(mod, x[1:m, 1:n], Bin) # binary variables assign jobs to machines
@variable(mod, z[1:m], Bin) # binary variables tell us which machines to use

# objective is to minimize cost
@objective(mod, Min, sum(c[i,j]*x[i,j] for i in 1:m for j in 1:n)
            + sum(h[i]*z[i] for i in 1:m))
    
# use at most b[i] units of capacity on each machine i
@constraint(mod, capacity[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i])
@constraint(mod, jobassign[j in 1:n], sum(x[i,j] for i in 1:m) == 1)
# Fixed cost logic
@constraint(mod, logic[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i]*z[i])

println("Solution Time with Gurobi Solver:")
@time(optimize!(mod))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-17
Solution Time with Gurobi Solver:
  3.860476 seconds (1.48 M allocations: 98.900 MiB, 0.76% gc time, 26.85% compilation time)


In [3]:
mod = Model(Cbc.Optimizer)
set_optimizer_attribute(mod, "LogLevel", 0)

@variable(mod, x[1:m, 1:n], Bin) # binary variables assign jobs to machines
@variable(mod, z[1:m], Bin) # binary variables tell us which machines to use

# objective is to minimize cost
@objective(mod, Min, sum(c[i,j]*x[i,j] for i in 1:m for j in 1:n)
            + sum(h[i]*z[i] for i in 1:m))
    
# use at most b[i] units of capacity on each machine i
@constraint(mod, capacity[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i])
@constraint(mod, jobassign[j in 1:n], sum(x[i,j] for i in 1:m) == 1)
# Fixed cost logic
@constraint(mod, logic[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i]*z[i])

println("Solution Time with Cbc Solver:")
@time(optimize!(mod))

Solution Time with Cbc Solver:
 50.721571 seconds (3.77 M allocations: 253.227 MiB, 0.24% gc time, 4.70% compilation time)
Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)


### Answer: test (b) 

In [4]:
mod = Model(Gurobi.Optimizer)
set_optimizer_attribute(mod, "OutputFlag", 0)

@variable(mod, x[1:m, 1:n], Bin) # binary variables assign jobs to machines
@variable(mod, z[1:m], Bin) # binary variables tell us which machines to use
@variable(mod, y, Bin) # binary variable that activates condition 3 if both condition 1 and condition 2 are met

# objective is to minimize cost
@objective(mod, Min, sum(c[i,j]*x[i,j] for i in 1:m for j in 1:n)
            + sum(h[i]*z[i] for i in 1:m))
    
# use at most b[i] units of capacity on each machine i
@constraint(mod, capacity[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i])
@constraint(mod, jobassign[j in 1:n], sum(x[i,j] for i in 1:m) == 1)
# Fixed cost logic
@constraint(mod, logic[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i]*z[i])
# condition 123
@constraint(mod, sum(z[i] for i in 1:10) >= 3y)
@constraint(mod, sum(z[i] for i in 11:20) >= 3y)
@constraint(mod, sum(z[i] for i in 21:30) <= 2 + 8(1-y))

println("Solution Time with Gurobi Solver:")
@time(optimize!(mod))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-17
Solution Time with Gurobi Solver:
  3.342600 seconds (34.05 k allocations: 2.611 MiB, 1.03% compilation time)


In [5]:
mod = Model(Cbc.Optimizer)
set_optimizer_attribute(mod, "LogLevel", 0)

@variable(mod, x[1:m, 1:n], Bin) # binary variables assign jobs to machines
@variable(mod, z[1:m], Bin) # binary variables tell us which machines to use
@variable(mod, y, Bin) # binary variable that activates condition 3 if both condition 1 and condition 2 are met

# objective is to minimize cost
@objective(mod, Min, sum(c[i,j]*x[i,j] for i in 1:m for j in 1:n)
            + sum(h[i]*z[i] for i in 1:m))
    
# use at most b[i] units of capacity on each machine i
@constraint(mod, capacity[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i])
@constraint(mod, jobassign[j in 1:n], sum(x[i,j] for i in 1:m) == 1)
# Fixed cost logic
@constraint(mod, logic[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i]*z[i])
# condition 123
@constraint(mod, sum(z[i] for i in 1:10) >= 3y)
@constraint(mod, sum(z[i] for i in 11:20) >= 3y)
@constraint(mod, sum(z[i] for i in 21:30) <= 2 + 8(1-y))

println("Solution Time with Cbc Solver:")
@time(optimize!(mod))

Solution Time with Cbc Solver:
 47.987502 seconds (138.78 k allocations: 11.101 MiB, 0.23% compilation time)
Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)


### Answer: test (c) 

In [6]:
mod = Model(Gurobi.Optimizer)
set_optimizer_attribute(mod, "OutputFlag", 0)
set_optimizer_attribute(mod, "TimeLimit", 100) # Set a time limit of 300 seconds
set_optimizer_attribute(mod, "MipFocus", 1)    # Focus on finding feasible solutions

@variable(mod, x[1:m, 1:n], Bin) # binary variables assign jobs to machines
@variable(mod, z[1:m], Bin) # binary variables tell us which machines to use
@variable(mod, T >= 0) # continuous variable representing the makespan

# objective is to minimize cost
@objective(mod, Min, T)
    
# use at most b[i] units of capacity on each machine i
@constraint(mod, capacity[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i])
@constraint(mod, jobassign[j in 1:n], sum(x[i,j] for i in 1:m) == 1)
# Fixed cost logic
@constraint(mod, logic[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i]*z[i])
# Makespan constraint
@constraint(mod, makespan[i in 1:m], sum(d[j]*x[i,j] for j in 1:n) <= T)
# Machine usage constraint
@constraint(mod, sum(z[i] for i in 1:m) <= 15)

println("Solution Time with Gurobi Solver:")
@time(optimize!(mod))

# Check if the solver found an optimal solution
if termination_status(mod) == MOI.OPTIMAL
    println("Optimal solution found.")
elseif termination_status(mod) == MOI.TIME_LIMIT
    println("Time limit reached. Best feasible solution found.")
else
    println("Solver did not converge to an optimal solution.")
end


Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-17
Solution Time with Gurobi Solver:
Set parameter MIPFocus to value 1
Set parameter TimeLimit to value 100
100.160768 seconds (356.38 k allocations: 14.212 MiB, 0.15% gc time, 0.15% compilation time)
Time limit reached. Best feasible solution found.


In [7]:
mod = Model(Cbc.Optimizer)
set_optimizer_attribute(mod, "LogLevel", 0)
set_optimizer_attribute(mod, "Seconds", 100) # Set a time limit of 300 seconds

@variable(mod, x[1:m, 1:n], Bin) # binary variables assign jobs to machines
@variable(mod, z[1:m], Bin) # binary variables tell us which machines to use
@variable(mod, T >= 0) # continuous variable representing the makespan

# objective is to minimize cost
@objective(mod, Min, T)
    
# use at most b[i] units of capacity on each machine i
@constraint(mod, capacity[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i])
@constraint(mod, jobassign[j in 1:n], sum(x[i,j] for i in 1:m) == 1)
# Fixed cost logic
@constraint(mod, logic[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i]*z[i])
# Makespan constraint
@constraint(mod, makespan[i in 1:m], sum(d[j]*x[i,j] for j in 1:n) <= T)
# Machine usage constraint
@constraint(mod, sum(z[i] for i in 1:m) <= 15)

println("Solution Time with Cbc Solver:")
@time(optimize!(mod))

# Check if the solver found an optimal solution
if termination_status(mod) == MOI.OPTIMAL
    println("Optimal solution found.")
elseif termination_status(mod) == MOI.TIME_LIMIT
    println("Time limit reached. Best feasible solution found.")
else
    println("Solver did not converge to an optimal solution.")
end

Solution Time with Cbc Solver:
101.279262 seconds (531.27 k allocations: 36.920 MiB, 0.02% gc time, 0.40% compilation time)
Time limit reached. Best feasible solution found.
Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -LogLevel 0 -Seconds 100 -solve -quit (default strategy 1)


### Answer: (d)
Testing Gurobi and Cbc solvers revealed significant differences in solution times. Gurobi consistently outperformed Cbc, solving problems much faster, while Cbc was 10-20 times slower, despite using less memory. Overall, Gurobi proved to be the most effective solver. However, Cbc remains a viable option for smaller problems or when a commercial solver is unavailable.

## Problem 2 -- Paint Cleaning

Your friend has decided to start a side hustle as a freelance painter. They are using a set of $n$ different paint batches, which they produce using a single paint mixer. The mixer needs to be cleaned between every pair of batches produced. 

The mixer cleaning times depend of the colors and the paint types.  For example, a long cleaning period is required if an oil-based paint is produced after a water-based paint, or to produce white paint after a
dark color.  The times are given in minutes in the following table, where where each entry $(i,j)$ denotes the cleaning time between batch $i$ and batch $j$.  It also takes some amount of time $d_i$ to mix each batch, given in the second table below. 

\begin{array}{c|cccccccccc}
    & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 \\
    \hline
    1  & 0  & 11 & 7  & 13 & 11 & 12 & 4  & 9  & 7  & 11 \\
    2  & 5  & 0  & 13 & 15 & 15 & 6  & 8  & 10 & 9  & 8  \\
    3  & 13 & 15 & 0  & 23 & 11 & 11 & 16 & 18 & 5  & 7  \\
    4  & 9  & 13 & 5  & 0  & 3  & 8  & 10 & 12 & 14 & 5  \\
    5  & 3  & 7  & 7  & 7  & 0  & 9  & 10 & 11 & 12 & 13 \\
    6  & 10 & 6  & 3  & 4  & 14 & 0  & 8  & 5  & 11 & 12 \\
    7  & 4  & 6  & 7  & 3  & 13 & 7  & 0  & 10 & 4  & 6  \\
    8  & 7  & 8  & 9  & 9  & 12 & 11 & 10 & 0  & 10 & 9  \\
    9  & 9  & 14 & 8  & 4  & 9  & 6  & 10 & 8  & 0  & 12 \\
    10 & 11 & 17 & 11 & 6  & 10 & 4  & 7  & 9  & 11 & 0  \\
\end{array}
 
|paint|mix time|
|--|--|
|1|40|
|2|35|
|3|45|
|4|32|
|5|50|
|6|42|
|7|44|
|8|30|
|9|33|
|10|55|


Since your friend has other obligations, they want to deal with this weekly production in the shortest possible time (mixing and cleaning).  What is the optimal order of paint batches to mix?  The order will be applied every week, so the cleaning time between the last batch of one week and the first of the following week needs to be accounted for in the total duration of cleaning.  

Solve this problem as a TSP. You may use either MTZ reformulation or subtour elimination (or both, for fun!).

### Answer:

In [8]:
# Miller-Tucker-Zemlin formulation
# Define the problem data (panits and cleaning time)
using JuMP, NamedArrays, Gurobi

panits = [:1, :2, :3, :4, :5, :6, :7, :8, :9, :10]
clean_time = [0 11 7 13 11 12 4 9 7 11;
                5 0 13 15 15 6 8 10 9 8;
                13 15 0 23 11 11 16 18 5 7;
                9 13 5 0 3 8 10 12 14 5;
                3 7 7 7 0 9 10 11 12 13;
                10 6 3 4 14 0 8 5 11 12;
                4 6 7 3 13 7 0 10 4 6;
                7 8 9 9 12 11 10 0 10 9;
                9 14 8 4 9 6 10 8 0 12 ;
                11 17 11 6 10 4 7 9 11 0]

mix_time = [40, 35, 45, 32, 50, 42, 44, 30, 33, 55]

t = NamedArray(clean_time,(panits,panits))
total_mix_time = sum(mix_time)
# print(total_mix_time)
N = size(clean_time,1)

# HELPER FUNCTION: returns the cycle containing the panits START.
function getSubtour(x,start)
    subtour = [start]
    while true
        j = subtour[end]
        for k in panits
            if x[k,j] == 1
                push!(subtour,k)
                break
            end
        end
        if subtour[end] == start
            break
        end
    end
    return subtour
end

# HELPER FUNCTION: returns a list of all cycles
function getAllSubtours(x)
    nodesRemaining = panits
    subtours = []
    while length(nodesRemaining) > 0
        subtour = getSubtour(x,nodesRemaining[1])
        push!(subtours, subtour)
        nodesRemaining = setdiff(nodesRemaining,subtour)
    end
    return subtours
end

getAllSubtours (generic function with 1 method)

In [9]:
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

@variable(m, x[panits, panits], Bin)                                      # must formulate as IP this time
@constraint(m, c1[j in panits], sum( x[i,j] for i in panits ) == 1)      # one after paint
@constraint(m, c2[i in panits], sum( x[i,j] for j in panits ) == 1)      # one before paint
@constraint(m, c3[i in panits], x[i,i] == 0 )                            # no self-loops
@objective(m, Min, sum( x[i,j]*t[i,j] for i in panits, j in panits ))   # minimize total clean time
                                    
# MTZ variables and constraints
@variable(m, u[panits])
@constraint(m, logic[i in panits, j in panits[2:end]], u[i] - u[j] + N*x[i,j] <= N-1 )

optimize!(m)
xx = value.(x)
subtours = getAllSubtours(xx) 
print("The optimal order of paint batches to mix: ", subtours, "\n")
# display(subtours)
println("Total time: ", objective_value(m) + total_mix_time)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-17
The optimal order of paint batches to mix: Any[[1, 5, 4, 9, 3, 6, 10, 8, 2, 7, 1]]
Total time: 457.0
